# The simplex method 

* Executes the simplex method as described in Rader (2010), using Dantzig's rule.


* To run a code cell:
    1. Click inside a code cell.
    2. Either press <key><i class="fa fa-step-forward" aria-hidden="true"></i></key> in the toolbar, or press <key>Ctrl</key> + <key>Enter</key>.

## Setup code. You only need to run this code cell once.

In [1]:
from sympy import symbols, solve, latex 
from sympy.matrices import Matrix, zeros
from sympy.interactive import printing
from IPython.display import Markdown, display
printing.init_printing(use_latex="mathjax")

def printmd(string):
    display(Markdown(string))

## Input the variable names of the canonical LP you want to solve.

* Make sure you change the variable names in all three places.

In [8]:
x_1, x_2, x_3, x_4, x_5 = symbols('x_1, x_2, x_3, x_4, x_5')
variables = [x_1, x_2, x_3, x_4, x_5] 

## Input the objective function vector $\mathbf{c}$, the constraint matrix $\mathbf{A}$, and the constraint vector $\mathbf{b}$ of the canonical LP you want to solve.

In [9]:
A = Matrix([[1, 5, 0, 1, 0],
            [-2, 3, 4, 0, 1]])
b = Matrix([1, 2])
c = Matrix([3, 2, 1, 0, 0.25])
maximize = False

## Input the initial basis.

In [10]:
basis = {x_2, x_3}

## Run the simplex method!

In [11]:
# Determine m and n 
(m, n) = A.shape

# Compute initial BFS
x = zeros(n, 1)
for i, var in enumerate(variables):
    if var in basis:
        x[i] = var

soln = solve(A*x - b)
bfs = zeros(n, 1)
for i, var in enumerate(variables):
    if var in basis:
        bfs[i] = soln[var]

# Print the initial BFS and basis
printmd("The intial BFS and basis are")
printmd("&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{{x}}^{{0}} = {0}$&nbsp;&nbsp;&nbsp;&nbsp;$\mathcal{{B}}^{{0}} = {1}$".format(latex(bfs.T), latex(basis)))

# Start main loop
iteration = 0
optimal_solution_found = False
unbounded = False

while True:
    iteration += 1
    printmd("**Iteration {0}**".format(iteration))
    
    # Compute the nonbasic variables
    # Sort the nonbasic variables according to the original variable order
    #   for more sensible printing
    nonbasic_variables = list(set(variables) - basis)
    nonbasic_variables.sort(key=lambda x: variables.index(x))
    
    # Compute simplex directions and reduced costs
    direction = {}
    reduced_cost = {}
    for nb in nonbasic_variables:
        # Set up simplex direction template
        d = zeros(n, 1)
        for i, var in enumerate(variables):
            if var in basis:
                d[i] = var
            elif var == nb:
                d[i] = 1
        
        # Solve for simplex direction
        soln = solve(A*d)
        direction[nb] = zeros(n,1)
        for i, var in enumerate(variables):
            if var in basis:
                direction[nb][i] = soln[var]
            elif var == nb:
                direction[nb][i] = 1
        
        # Compute reduced cost
        reduced_cost[nb] = c.dot(direction[nb])
        
    # Print simplex directions and associated reduced costs
    printmd("&nbsp;&nbsp;&nbsp;&nbsp;The simplex directions are:")
    for nb in nonbasic_variables:
        printmd("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{{d}}^{{{0}}} = {1}$&nbsp;&nbsp;&nbsp;&nbsp;$\\bar{{c}}_{{{0}}} = {2}$".format(nb, latex(direction[nb].T), reduced_cost[nb]))
        
    # Choose entering variable according to Dantzig's rule
    if maximize:
        (entering, max_reduced_cost) = max(reduced_cost.items(), key=(lambda x: x[1]))
        if max_reduced_cost <= 0:
            optimal_solution_found = True
            break
    else:
        (entering, min_reduced_cost) = min(reduced_cost.items(), key=(lambda x: x[1]))
        if min_reduced_cost >= 0:
            optimal_solution_found = True
            break
    
    # Print entering variable
    printmd("&nbsp;&nbsp;&nbsp;&nbsp;Using Dantzig's rule, the entering variable is ${0}$.".format(latex(entering)))
            
    # Check for unboundedness
    if all(direction[entering][j] >= 0 for j, var in enumerate(variables)):
        unbounded = True
        break
    
    # Compute step size and leaving variable using MRT
    ratio = {}
    for j, var in enumerate(variables):
        if direction[entering][j] < 0:
            ratio[var] = bfs[j] / -direction[entering][j]
    (leaving, lambda_max) = min(ratio.items(), key=(lambda x: x[1]))
    
    # Print the step size and leaving variable
    printmd("&nbsp;&nbsp;&nbsp;&nbsp;Using the minimum ratio test, the step size is $\lambda_{{\max}} = {0}$ and the leaving variable is ${1}$.".format(lambda_max, latex(leaving)))
        
    # Update solution and basis
    bfs = bfs + lambda_max * direction[entering]
    basis.add(entering)
    basis.remove(leaving)
    
    # Print the next BFS and basis
    printmd("&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{{x}}^{{{0}}} = {1}$&nbsp;&nbsp;&nbsp;&nbsp;$\mathcal{{B}}^{{{0}}} = {2}$".format(iteration, latex(bfs.T), latex(basis)))

# Print the reason for termination
if optimal_solution_found:
    printmd("&nbsp;&nbsp;&nbsp;&nbsp;Since there are no improving simplex directions, the BFS $\mathbf{{x}}^{{{0}}} = {1}$ is optimal with value {2}.".format(iteration - 1, latex(bfs.T), c.dot(bfs)))
elif unbounded:
    printmd("&nbsp;&nbsp;&nbsp;&nbsp;Since $\mathbf{{d}}^{{{0}}}$ is an improving simplex direction with all nonnegative components, the LP is unbounded.".format(latex(entering)))
else:
    print("ERROR. Something went wrong: algorithm terminated, but optimal solution or certificate of unboundedness were not found")

The intial BFS and basis are

&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{x}^{0} = \left[\begin{matrix}0 & \frac{1}{5} & \frac{7}{20} & 0 & 0\end{matrix}\right]$&nbsp;&nbsp;&nbsp;&nbsp;$\mathcal{B}^{0} = \left\{x_{2}, x_{3}\right\}$

**Iteration 1**

&nbsp;&nbsp;&nbsp;&nbsp;The simplex directions are:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{d}^{x_1} = \left[\begin{matrix}1 & - \frac{1}{5} & \frac{13}{20} & 0 & 0\end{matrix}\right]$&nbsp;&nbsp;&nbsp;&nbsp;$\bar{c}_{x_1} = 13/4$

...

&nbsp;&nbsp;&nbsp;&nbsp;The simplex directions are:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{d}^{x_1} = \left[\begin{matrix}1 & 0 & \frac{1}{2} & -1 & 0\end{matrix}\right]$&nbsp;&nbsp;&nbsp;&nbsp;$\bar{c}_{x_1} = 7/2$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{d}^{x_2} = \left[\begin{matrix}0 & 1 & - \frac{3}{4} & -5 & 0\end{matrix}\right]$&nbsp;&nbsp;&nbsp;&nbsp;$\bar{c}_{x_2} = 5/4$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{d}^{x_5} = \left[\begin{matrix}0 & 0 & - \frac{1}{4} & 0 & 1\end{matrix}\right]$&nbsp;&nbsp;&nbsp;&nbsp;$\bar{c}_{x_5} = 0$

&nbsp;&nbsp;&nbsp;&nbsp;Since there are no improving simplex directions, the BFS $\mathbf{x}^{1} = \left[\begin{matrix}0 & 0 & \frac{1}{2} & 1 & 0\end{matrix}\right]$ is optimal with value 1/2.